In [0]:
# for box detaction

import os
import time
import statistics
import cv2
from scipy.stats import mode

# for image preprocess
import pandas as pd
from PIL import Image,ImageFilter
import numpy as np
import sys
import csv
import PIL.ImageOps

# for model
# %tensorflow_version 1.x  # For colab
import tensorflow
from tqdm import tqdm
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
labels=pd.read_excel("/content/gdrive/My Drive/Colab Notebooks/results/Inference/Labels Inference - 47 Labels.xlsx",index_col="Sr. No.")

In [62]:
#print("Loading Model")
model = load_model("/content/gdrive/My Drive/Colab Notebooks/model_letters_digits_space_sc_88.h5")

In [0]:
# # image pre process

# def preprocess(data,img,num_images):

#   # Convert cv2 image to PIL image
#   #cv2_img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
#   image = Image.fromarray(img)
#   image = image.crop((4, 3, image.size[0]-4, image.size[0]-4))     # im.crop((left, top, right, bottom)) 
#   img_resized = image.resize((78,78))
#   img_resized=PIL.ImageOps.invert(img_resized)
#   # img_file = img_resized.filter(ImageFilter.MedianFilter())
#   threshold = 80
#   img_file = img_resized.point(lambda p: p > threshold and 255)

#   # Save Greyscale values
#   value = np.asarray(img_file.getdata(), dtype=np.int).reshape((img_file.size[1], img_file.size[0]))
#   value = value.flatten()
  
#   # To improve image quality on 4 top and bottom lines
#   for i in range (0,4):
#     if (mode(value[78*i:78*(i+1)])[0]>200 or (mode(value[78*i:78*(i+1)])[0] < 80 and mode(value[78*i:78*(i+1)])[1][0] < 54)):
#         value[78*i:78*(i+1)]=0
#     if (mode(value[-78*(i+1):-78*(i)-1])[0]>200 or (mode(value[-78*(i+1):-78*(i)-1])[0] < 80 and mode(value[-78*(i+1):-78*(i)-1])[1][0] < 54)):
#         value[-78*(i+1):-78*(i)-1]=0
#   data.loc[num_images] = value
#   #print(value)
#   num_images += 1
#   #print("image preprocessed")
#   return (data,num_images)

In [0]:
# image pre process

def preprocess(img,value):

  # Convert cv2 image to PIL image
  #cv2_img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
  image = Image.fromarray(img)
  image = image.crop((4, 3, image.size[0]-4, image.size[0]-4))     # im.crop((left, top, right, bottom)) 
  img_resized = image.resize((78,78))
  img_resized=PIL.ImageOps.invert(img_resized)
  # img_file = img_resized.filter(ImageFilter.MedianFilter())
  threshold = 80
  img_file = img_resized.point(lambda p: p > threshold and 255)

  # Save Greyscale values
  value = np.asarray(img_file.getdata(), dtype=np.int).reshape((img_file.size[1], img_file.size[0]))
  value = value.flatten()
  
  # To improve image quality on 4 top and bottom lines
  for i in range (0,4):
    if (mode(value[78*i:78*(i+1)])[0]>200 or (mode(value[78*i:78*(i+1)])[0] < 80 and mode(value[78*i:78*(i+1)])[1][0] < 54)):
      value[78*i:78*(i+1)]=0
    if (mode(value[-78*(i+1):-78*(i)-1])[0]>200 or (mode(value[-78*(i+1):-78*(i)-1])[0] < 80 and mode(value[-78*(i+1):-78*(i)-1])[1][0] < 54)):
      value[-78*(i+1):-78*(i)-1]=0
  return(value)

In [0]:
# Image prediction

def inference(test,count):
  test_image = []

  # Saivng all images in X_test
  for i in tqdm(range(test.shape[0])):
    im_buf = test.values[i][0:]
    im_array = np.int8(np.reshape(im_buf, (78, 78))) 
    #img = image.load_img(train.iloc[:,1:].as_matrix().astype('str'), target_size=(64,64,1), color_mode='grayscale')
    img = image.img_to_array(im_array)
    img = img/255
    test_image.append(img)
  X_test = np.array(test_image)
  X_test = np.repeat(X_test, 3, -1)


  print("Predicting")
  pred=model.predict(X_test)
  # for i in pred:
  #     print(i)
      
  output=np.argmax(pred, axis=1)
  # print("Reading Lables")
  # labels=pd.read_excel("/content/gdrive/My Drive/Colab Notebooks/results/Inference/Labels Inference - 47 Lables.xlsx",index_col="Sr. No.")
  # print("out")
  col=['pred']
  section=out=pd.DataFrame(columns=['pred'])
  row=0
  for i in output:
    section.loc[row]=labels.iloc[i].values
    row+=1
    #print(labels.iloc[i].values)
  
  section.to_csv("/content/gdrive/My Drive/Colab Notebooks/results/Inference/section"+str(count)+".csv")

In [0]:
# Load the image in black and white (0 - b/w, 1 - color).
def inner_box(img,count,height):
  # img is image
  # Count is section 0-5
  # height is image height
  # number is image number from the batch

  #Invert the image to be white on black for compatibility with findContours function.
  imgray = 255 - img

# Using different combination of dilation and blur to get as much as contours possible

  # rects1
  
  kernel = np.ones((1,10),np.uint8)
  imgray1 = cv2.dilate(imgray,kernel,iterations = 1)
  kernel = np.ones((10,1),np.uint8)
  imgray1 = cv2.dilate(imgray1,kernel,iterations = 1)
  thresh1 = cv2.adaptiveThreshold(imgray1, 255, cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY,15,2)

  #Find all the contours in thresh. 
  contours, hierarchy = cv2.findContours(thresh1, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
  rects1 = [cv2.boundingRect(cnt) for cnt in contours]


  # rects2 imp
  imgray2 = cv2.blur(imgray,(10,3))
  thresh2 = cv2.adaptiveThreshold(imgray2, 255, cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY,15,2)

  
  #Find all the contours in thresh. 
  contours, hierarchy = cv2.findContours(thresh2, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
  rects2 = [cv2.boundingRect(cnt) for cnt in contours]

  # rects3 imp
  imgray3 = cv2.blur(imgray,(3,10))
  thresh3 = cv2.adaptiveThreshold(imgray3, 255, cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY,15,2)
  
  #Find all the contours in thresh. 
  contours, hierarchy = cv2.findContours(thresh3, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
  rects3 = [cv2.boundingRect(cnt) for cnt in contours]

  #rects4 imp
  kernel = np.ones((2,7),np.uint8)
  imgray4 = cv2.dilate(imgray,kernel,iterations = 1)
  thresh4 = cv2.adaptiveThreshold(imgray4, 255, cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY,15,2)
  
  #Find all the contours in thresh.
  contours, hierarchy = cv2.findContours(thresh4, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
  rects4 = [cv2.boundingRect(cnt) for cnt in contours]

  
  #rects5 imp
  kernel = np.ones((7,2),np.uint8)
  imgray5 = cv2.dilate(imgray,kernel,iterations = 1)
  thresh5 = cv2.adaptiveThreshold(imgray5, 255, cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY,15,2)
  
  #Find all the contours in thresh.
  contours, hierarchy = cv2.findContours(thresh5, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
  rects5 = [cv2.boundingRect(cnt) for cnt in contours]
  
  
# Defining function to filter sections for different size sections

  def filterrects1(raw_rects,rects):
      for i in raw_rects:
          if (abs(i[2]-i[3])<10):
              if (i[2]>height/75 and i[2]<height/40 and i[3]>height/75 and i[3]<height/40 and i not in rects):
                  rects.append(i)
      return(rects)
  
  def filterrects2(raw_rects,rects):
      for i in raw_rects:
          if (i[2]>height/40 and i[2]<height/25 and i[3]>height/40 and i[3]<height/25 and i not in rects):
              rects.append(i)
      return(rects)
  
# checking the section number and calling function accordingly.
  if count!=0:
      rects=[]
      rects=filterrects1(rects1,rects)
      rects=filterrects1(rects2,rects)
      rects=filterrects1(rects3,rects)
      rects=filterrects1(rects4,rects)
      rects=filterrects1(rects5,rects)
                  
  else:
      rects=[]
      rects=filterrects2(rects1,rects)
      rects=filterrects2(rects2,rects)
      rects=filterrects2(rects3,rects)
      rects=filterrects2(rects4,rects)
      rects=filterrects2(rects5,rects)

  rects_final=rects.copy()
  
# Removing the sections which are overlapping
  for i in range(0,len(rects)-1):
      try:
          for j in range(i+1,len(rects)):
              if (abs(rects[i][0] - rects[j][0]) < 15 and abs(rects[i][1] - rects[j][1]) < 15):
                  if (rects[i][2] < rects[j][2] and rects[i][3] < rects[j][3]):
                      rects_final.remove(rects[j])
                  else:
                      rects_final.remove(rects[i])
      except:
          continue
#     print(len(rects_final))
  rects=rects_final.copy()

  #Calculate the combined bounding rectangle points.
  top_x = min([x for (x, y, w, h) in rects])
  top_y = min([y for (x, y, w, h) in rects])
  bottom_x = max([x+w for (x, y, w, h) in rects])
  bottom_y = max([y+h for (x, y, w, h) in rects])


  from scipy.stats import mode
  h_mode = mode([h for (x, y, w, h) in rects])[0]
  #w_mode = mode([w for (x, y, w, h) in rects])[0]
  n=int(((bottom_y-top_y)/h_mode))

# Sorting the sections in increasing order as they occur in image
  import operator
  temp=[]
  b = [[] for i in range(0, n+3)]
  for j in range (1,n+3):
    for i in rects:
        x=i[0]
        y=i[1]
        w=i[2]
        h=i[3]
          
    # Keeping constraint on y and sorting as per x pixels
        if ((y > bottom_y-h_mode*j-h_mode/2.5) and (i not in temp)):
          b[j].append(i)
          temp.append(i)
        b[j]=sorted(b[j],key=lambda x:(-x[0]))
  #b.pop(0)
  tar_dir=r"/content/gdrive/My Drive/Colab Notebooks/Output/Inner/"+str(2)+"/"+str(count)+"/"
  if not os.path.exists(tar_dir):
      #print("here2")
    os.makedirs(tar_dir) 

# Traversing through each section and saving and predicting each contour
  count1=0

# dataframe to store all images values
  columnNames=[]
  for i in range (0,6084):
    columnNames.append(i)
  print("count is",count)
  data=pd.DataFrame(columns = columnNames)
  num_images=0
  for i in b:
    for j in i:
      try:
        x=j[0]
        y=j[1]
        w=j[2]
        h=j[3]

#To append image in dataframe
        value=[]
        cv2.imwrite(tar_dir+str(count1)+".jpg",img[y-2:y+h+2,x-2:x+w+2])
        value=preprocess(img[y-2:y+h+2,x-2:x+w+2],value)
        #data,num_images=preprocess(data,img[y-2:y+h+2,x-2:x+w+2],num_images)
        data.loc[num_images]=value
        num_images += 1
        time.sleep(0.01)
        count1+=1
      except:
        continue
# To Infer the datframe boxes
    #print("inferecing")
  # if data.empty == True:
  #   continue
  # else:
  #   #print(data.head())
  print(data.shape)
  inference(data,count)

In [0]:
# to check if all major boxes came and try to get correct one by hard code if some error.

def validrects(rects3):
    if len (rects3)!=6:
        rects_f=[]
        
        # saving all cooredinates of large box by manuualy calculating x,y,w,h using height of whole image and 1st box
        first=rects3[0]
        second=(first[0],first[1]+first[3]+round(height/159.41),round(first[2]*2.05),round(height/11.13))
        third=(first[0],second[1]+second[3]+round(height/159.41),round(first[2]*2.05),round(height/4.45))
        fourth=(first[0]+(round(first[2]*2.05*5/35)),third[1]+third[3]+round(height/159.41),round(first[2]*2.05),round(height/8.37))
        fifth=(first[0]+(round(first[2]*2.05*5/35)),fourth[1]+fourth[3]+round(height/159.41),round(first[2]*2.05),round(height/11.89))
        sixth=(first[0]+(round(first[2]*2.05*4/35)),fifth[1]+fifth[3]+round(height/159.41),round(first[2]*2.05),round(height/8.77))
        rects_f=[first,second,third,fourth,fifth,sixth]
        return rects_f
    
    elif (len (rects3)==6 and  rects3[1][3]/rects3[0][3] > 1.25 and rects3[2][3]/rects3[0][3] > 3.25 and 
    rects3[3][3]/rects3[0][3] > 1.50 and rects3[4][3]/rects3[0][3] > 1.25 and rects3[5][3]/rects3[0][3] > 1.50):
        return rects3
    else:
        rects_f=[]
        # saving all cooredinates of large box by manuualy calculating x,y,w,h using height of whole image and 1st box
        first=rects3[0]
        second=(first[0],first[1]+first[3]+round(height/159.41),round(first[2]*2.05),round(height/11.13))
        third=(first[0],second[1]+second[3]+round(height/159.41),round(first[2]*2.05),round(height/4.45))
        fourth=(first[0]+(round(first[2]*2.05*5/35)),third[1]+third[3]+round(height/159.41),round(first[2]*2.05),round(height/8.37))
        fifth=(first[0]+(round(first[2]*2.05*5/35)),fourth[1]+fourth[3]+round(height/159.41),round(first[2]*2.05),round(height/11.89))
        sixth=(first[0]+(round(first[2]*2.05*4/35)),fifth[1]+fifth[3]+round(height/159.41),round(first[2]*2.05),round(height/8.77))
        rects_f=[first,second,third,fourth,fifth,sixth]
        return rects_f
        return rects3
        

In [67]:
#Load the image in black and white (0 - b/w, 1 - color).
number=2
img = cv2.imread(r'/content/gdrive/My Drive/Colab Notebooks/results/Inference/form/'+str(number)+'.jpg', 0)

#Get the height and width of the image.
height, width = img.shape[:2]

#Invert the image to be white on black for compatibility with findContours function.
imgray = 255 - img

# Remove noise using dilate and thershold
thresh = cv2.adaptiveThreshold(imgray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,15,2)
kernel = np.ones((2,2),np.uint8)
thresh = cv2.dilate(thresh,kernel,iterations = 1)

#Find all the contours in thresh. 
contours, hierarchy = cv2.findContours(thresh, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)

#Find bounding rectangles for each contour.
rects = [cv2.boundingRect(cnt) for cnt in contours]
rects2=[]
count=0
for i in rects:
    # hierarchy [Next, Previous, First_Child, Parent]
    if (hierarchy[0][count][2]>0 and hierarchy[0][count][3]<0):   # condn that contour is parent and have child.
        if (i[2]>thresh.shape[1]/3 and i[2] < thresh.shape[1]):   # condn on width of contours
            rects2.append(i)
    count+=1
h_mode = mode([h for (x, y, w, h) in rects2])[0]

# to remove noisy contour i.e. which are small in height if these
if h_mode < height/20:
    rects2=[]
    count=0
    for i in rects:
        if (hierarchy[0][count][2]>0):
            if (i[2]>thresh.shape[1]/3 and i[2] < thresh.shape[1] and i[3] > h_mode*1.2 ):
                rects2.append(i)
        count+=1

# to remove duplicate contours having very less coordinate difference
rects3=rects2.copy()
for i in range(0,len(rects2)-1):
    try:
        for j in range(i+1,len(rects2)):
            if (rects2[i][1] - rects2[j][1] <50):
                if rects2[i][2]>rects2[j][2]:
                    value=rects2[j]
                    rects3.remove(value)
                else :
                    value=rects2[i]
                    rects3.remove(value)
    except:
        continue

# sort in correct order
rects3=sorted(rects3,key=lambda x:(x[1]))
rects3=validrects(rects3)
# saving all
count=0
#print(len(rects3))
# To fetch inner box
for j in rects3:
  # try:
    x=j[0]
    y=j[1]
    w=j[2]
    h=j[3]
    if not os.path.exists(r"/content/gdrive/My Drive/Colab Notebooks/Output/Outer/"+str(number)):
      os.makedirs(r"/content/gdrive/My Drive/Colab Notebooks/Output/Outer/"+str(number)) 
    cv2.imwrite(r"/content/gdrive/My Drive/Colab Notebooks/Output/Outer/"+str(number)+"/img"+str(count)+".jpg",img[y:y+h,x:x+w])
    inner_box(img[y:y+h,x:x+w],count,height)
    #print(count)
    time.sleep(0.001)
    count+=1
  # except:
  #   continue

count is 0


100%|██████████| 6/6 [00:00<00:00, 1925.61it/s]

(6, 6084)
Predicting


count is 1


100%|██████████| 87/87 [00:00<00:00, 2519.41it/s]

...

100%|██████████| 30/30 [00:00<00:00, 2565.32it/s]

(30, 6084)
Predicting


count is 5


100%|██████████| 55/55 [00:00<00:00, 2871.85it/s]

(55, 6084)
Predicting
